In [2]:
import tangos
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import matplotlib
import pynbody
import matplotlib as mpl
import os

In [3]:
# The simulation data is made of 1000s of unformatted fortran binaries 
# (not very useful I know, but these codes are primarily optimized for HPC performance rather than user convenience).
# The pynbody package (which you might need to pip install) will make sense of the data formart
# and allow you to convert it to data tables suiting your need

whole_simulation = pynbody.load("./output_00186/")

c:\Users\choij\AppData\Local\Programs\Python\Python313\Lib\site-packages\pynbody\snapshot\ramses.py:833: UserWarning: Using field 'tform' to distinguish stars. If this is wrong, try editing your config.ini, section [ramses], entry particle-distinguisher.
  warnings.warn("Using field %s to distinguish stars. If this is wrong, try editing your config.ini, section [ramses], entry particle-distinguisher."%pb_name)


In [4]:
# The simulation itself is a large chunk of the Universe, with its large scale structure and cosmological filamentary structure
# You are welcome to explore it, but for the purpose of this simulation, we actually focussed all the computational power
# to produce a very well-resolved Milky Way galaxy in the centre
# I am now just centering on it and extracting its direct surroundings
pynbody.analysis.halo.center(whole_simulation.halos()[1])

<Transformation null, translate, offset_velocity>

In [5]:
galaxy = whole_simulation[pynbody.filt.Sphere(pynbody.array.SimArray(400, units='kpc'))]

# Explore our chosen object

In [ ]:
# This galaxy contains a certain number of star particles 
# In the context of these simulations (using smoothed particle hydrodynamics), you can view the word "particle"
# as equivalent to the simulation's "pixel" or resolution element. More particles == better-resolved simulations
galaxy.star

<FamilySubSnap "output_00186:filtered::star" len=787507>

In [9]:
# Gas particles
galaxy.gas

<FamilySubSnap "output_00186:filtered::gas" len=2920703>

In [10]:
# And dark matter particles
galaxy.dm

<FamilySubSnap "output_00186:filtered::dm" len=45975>

In [ ]:
# The data is organised into families. 
print(f"Here are all the families we are working with: {galaxy.families()}.")
print(f"Here are the keys for dark matter: {galaxy.dm.keys()}.")
print(f"Here are the keys for stars: {galaxy.star.keys()}.")
print(f"Here are the keys for gas particles: {galaxy.gas.keys()}.")
gas_density = galaxy.gas['rho']  # Gas density is a field that we have.
gas_temp = galaxy.gas['temp']  # Gas temperature is not one of the fields listed, however we can still refer to temperature, and pynbody will simply calculate it using physics!

Here are all the families we are working with: [<Family dm>, <Family star>, <Family gas>].
Here are the keys for dark matter: ['y', 'vx', 'z', 'x', 'mass', 'vel', 'vz', 'pos', 'vy'].
Here are the keys for stars: ['y', 'vx', 'z', 'x', 'mass', 'vel', 'vz', 'pos', 'vy'].
Here are the keys for gas particles: ['y', 'vx', 'z', 'x', 'mass', 'vel', 'vz', 'pos', 'vy', 'rho', 'p', 'metal_iron', 'metal_ox', 'smooth'].


c:\Users\choij\AppData\Local\Programs\Python\Python313\Lib\site-packages\pynbody\derived.py:229: UserWarning: No ionization fractions found, assuming fully ionised gas above 10^4 and neutral below 10^4K. This is a very crude approximation.
  warnings.warn("No ionization fractions found, assuming fully ionised gas above 10^4 and neutral below 10^4K. "


SimArray([513797.31015089, 268566.40811433, 850326.77127817, ...,
           14721.04130092, 619524.90781319,  16722.46935158],
         shape=(2920703,), 'K')

In [15]:
galaxy.star.keys()

['y', 'vx', 'z', 'x', 'mass', 'vel', 'vz', 'pos', 'vy']

In [14]:
# For each family of particles, I can ask for their individual physical quantities
# for example 'mass', 'x', 'y', 'z', 'vx', 'vy', 'vz' are accessible across all families
galaxy.star['mass'].in_units("Msol")

SimArray([5540.08414868, 5472.45568863, 5592.8093701 , ..., 6183.92323222,
          5335.8550178 , 5469.94751618], 'Msol')

In [15]:
galaxy.gas['mass'].in_units("g")

SimArray([1.54863227e+38, 1.41092754e+38, 1.64120492e+38, ...,
          1.57752009e+32, 4.79759321e+31, 7.92933290e+31], 'g')

In [16]:
galaxy.dm['mass'].in_units("kg")

SimArray([3.19521069e+36, 3.19521069e+36, 3.19521069e+36, ...,
          3.99401337e+35, 3.99401337e+35, 3.99401337e+35], 'kg')

In [21]:
# But not all physical quantities apply to all families
# The mass fraction of ozygen, for example, only applies to stars and gas
# (dark matter does not interact with visible matter like iron)
galaxy.star['metal_ox']

SimArray([5.49915161e-03, 4.39299327e-03, 4.40183729e-03, ...,
          8.66836074e-05, 6.81703136e-03, 1.57880677e-02])

In [23]:
# You can all available fields for each family with
galaxy.star.keys()

['vy',
 'y',
 'metal',
 'pos',
 'metals',
 'x',
 'vel',
 'metal_ox',
 'vz',
 'vx',
 'z',
 'mass']

## To convert stars to binary files:

import struct

# 1. Select your data
stars = galaxy.star
n_stars = len(stars)

# 2. Normalize Positions (fit into a unit box)
# Assuming you centered it already as per your existing cells
max_range = 400 # kpc, matching your Sphere filter
pos_normalized = stars['pos'] / max_range 

# 3. Select Metadata (e.g., Oxygen)
# Normalize metadata between 0 and 1 for easy coloring later
ox = stars['metal_ox']
ox_min, ox_max = np.min(ox), np.max(ox)
ox_normalized = (ox - ox_min) / (ox_max - ox_min)

# 4. Write to binary
# Format: float (x), float (y), float (z), float (oxygen)
print(f"Exporting {n_stars} stars...")

with open("galaxy_data.bytes", "wb") as f:
    for i in range(n_stars):
        # 'ffff' means 4 floats. Unity uses Left-Handed coords (x, y, z), 
        # Simulation usually Right-Handed. You might need to swap y/z or negate x later.
        # For now, let's just write x, y, z, data
        data = struct.pack('ffff', 
                           pos_normalized[i][0], 
                           pos_normalized[i][1], 
                           pos_normalized[i][2], 
                           ox_normalized[i])
        f.write(data)

print("Done!")